In [ ]:
import pandas as pd
import numpy as np
import os

# Asegurar carpeta de salida
os.makedirs("../data/processed", exist_ok=True)

ticker = "AAPL"
print(f"--- Procesando datos para {ticker} ---\n")

# 1. Cargar datos crudos
try:
    df_fin = pd.read_csv(f"../data/raw/{ticker}_financial_data.csv")
    df_mkt = pd.read_csv(f"../data/raw/{ticker}_market_data.csv")
    print(f"✅ Datos cargados: {len(df_fin)} reportes financieros, {len(df_mkt)} días de mercado")
except FileNotFoundError as e:
    print(f"❌ Error: {e}")
    print("Asegúrate de ejecutar primero el script de extracción.")
    exit()

# Convertir fechas
df_fin['Date'] = pd.to_datetime(df_fin['Date'])
df_mkt['Date'] = pd.to_datetime(df_mkt['Date'])

# --- PARTE A: DICCIONARIO DE TRADUCCIÓN (Paper PDF vs Yahoo) ---
print("\n📋 Mapeando variables del paper a columnas de Yahoo Finance...")

mapeo_contable = {
    # PDF Code : Yahoo Finance Column Name
    "at": "Total Assets",
    "lt": "Total Liabilities Net Minority Interest",
    "seq": "Stockholders Equity", 
    "sale": "Total Revenue",
    "cogs": "Cost Of Revenue",
    "ebit": "EBIT",
    "ni": "Net Income",
    "capx": "Capital Expenditure",
    "che": "Cash And Cash Equivalents",
    "inv": "Inventory",
    "rect": "Accounts Receivable"
}

# 2. Crear DataFrame Limpio con códigos del Paper
df_clean_fin = pd.DataFrame()
df_clean_fin['date_fin'] = df_fin['Date']

columnas_faltantes = []
columnas_mapeadas = []

for codigo_paper, columna_yahoo in mapeo_contable.items():
    try:
        if columna_yahoo in df_fin.columns:
            df_clean_fin[codigo_paper] = df_fin[columna_yahoo]
            columnas_mapeadas.append(codigo_paper)
        else:
            print(f"⚠️  '{codigo_paper}' ({columna_yahoo}) - NO ENCONTRADA")
            df_clean_fin[codigo_paper] = np.nan
            columnas_faltantes.append(codigo_paper)
    except Exception as e:
        print(f"❌ Error mapeando {codigo_paper}: {e}")
        df_clean_fin[codigo_paper] = np.nan
        columnas_faltantes.append(codigo_paper)

print(f"\n✅ Variables mapeadas exitosamente: {len(columnas_mapeadas)}/{len(mapeo_contable)}")
if columnas_faltantes:
    print(f"⚠️  Variables faltantes: {', '.join(columnas_faltantes)}")

# Calcular algunas variables derivadas comunes (opcionales)
print("\n🧮 Calculando variables derivadas básicas...")

# Equity derivado si falta (at - lt)
if pd.isna(df_clean_fin['seq']).all() and 'at' in df_clean_fin and 'lt' in df_clean_fin:
    df_clean_fin['seq'] = df_clean_fin['at'] - df_clean_fin['lt']
    print("   → 'seq' calculado como at - lt")

# CapEx a valor absoluto (viene negativo en cashflow)
if 'capx' in df_clean_fin.columns:
    df_clean_fin['capx'] = df_clean_fin['capx'].abs()

print("\n--- Datos Contables Traducidos (Primeras 3 filas) ---")
print(df_clean_fin.head(3).to_string())

# Estadísticas básicas
print("\n--- Estadísticas de Variables Contables ---")
print(df_clean_fin.describe().loc[['count', 'mean']].to_string())

# --- PARTE B: UNIÓN CON DATOS DE MERCADO ---
print("\n🔗 Uniendo datos de mercado con datos contables...")
print("   (Usando merge_asof: cada día de mercado toma el último reporte disponible)")

# Ordenar ambos DataFrames
df_mkt = df_mkt.sort_values("Date").reset_index(drop=True)
df_clean_fin = df_clean_fin.sort_values("date_fin").reset_index(drop=True)

# Implementar el lag de 4 meses del paper
# "Accounting variables available 4 months after fiscal period end"
df_clean_fin['date_disponible'] = df_clean_fin['date_fin'] + pd.DateOffset(months=4)

print(f"   Aplicando lag de 4 meses (disponibilidad de reportes)")
print(f"   Ejemplo: Reporte de {df_clean_fin['date_fin'].iloc[0].strftime('%Y-%m-%d')} "
      f"→ Disponible desde {df_clean_fin['date_disponible'].iloc[0].strftime('%Y-%m-%d')}")

# Merge asof con la fecha ajustada
df_final = pd.merge_asof(
    df_mkt, 
    df_clean_fin.drop('date_fin', axis=1),  # Eliminamos date_fin, solo usamos date_disponible
    left_on="Date", 
    right_on="date_disponible", 
    direction="backward"
)

# Estadísticas del merge
print(f"\n✅ Merge completado:")
print(f"   • Total días de mercado: {len(df_final)}")
print(f"   • Días con datos contables: {df_final['date_disponible'].notna().sum()}")
print(f"   • Días sin datos contables: {df_final['date_disponible'].isna().sum()}")

# Eliminar días sin datos contables (antes del primer reporte)
df_final_clean = df_final.dropna(subset=['date_disponible']).copy()
print(f"   • Días finales (con datos completos): {len(df_final_clean)}")

# Renombrar para claridad
df_final_clean.rename(columns={'Date': 'date_market', 'date_disponible': 'date_accounting'}, inplace=True)

# Reordenar columnas (fechas primero, luego precio, luego contabilidad)
cols_fecha = ['date_market', 'date_accounting']
cols_precio = ['Open', 'High', 'Low', 'Close', 'Volume']
cols_contables = [col for col in df_final_clean.columns if col not in cols_fecha + cols_precio]
df_final_clean = df_final_clean[cols_fecha + cols_precio + cols_contables]

# Guardar
output_path = f"../data/processed/{ticker}_ready_for_features.csv"
df_final_clean.to_csv(output_path, index=False)

print(f"\n✅ Datos procesados guardados en: {output_path}")
print(f"   Rango temporal: {df_final_clean['date_market'].min().strftime('%Y-%m-%d')} "
      f"a {df_final_clean['date_market'].max().strftime('%Y-%m-%d')}")
print(f"   Total de columnas: {len(df_final_clean.columns)}")
print(f"   Total de filas: {len(df_final_clean)}")

# Muestra final
print("\n--- Vista Previa de Datos Finales ---")
print(df_final_clean.head(3).to_string())